In [1]:

import dask
import dask.array as da
import torch
import numpy as np
tensor = torch.randn(501,300,20000)
chunks_size = (50,300,20000)
dask_array = da.from_array(tensor.numpy(), chunks=chunks_size)  
print(dask_array.shape)
import torch.nn as nn
model = nn.Sequential(nn.Flatten(),nn.Linear(in_features=300*200*100, out_features=10))  
def inference(chunk, model):
 
    model.eval()


    expected_shape = (chunk.shape[0], 300, 20000)
    assert chunk.shape == expected_shape, f"Invalid chunk shape: {chunk.shape}"

    with torch.no_grad():
        chunk_tensor = torch.from_numpy(chunk)
        chunk_flat = chunk_tensor.view(chunk_tensor.size(0), -1)
        output=model(chunk_flat)
        
    output=output.detach().numpy()
    output = output[:,np.newaxis]
    return output

result = dask_array.map_blocks(inference,model=model,dtype=np.float32,chunks = chunks_size)
result_array = result.compute()
#


(501, 300, 20000)


In [2]:
result_array.shape

(501, 1, 10)

In [4]:
result_serial = model(tensor)
result_serial.shape

torch.Size([500, 10])

In [4]:
!pip install h5py


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 29.8 MB/s eta 0:00:00m eta 0:00:010:01


In [4]:
import h5py
import torch

# Suppose your tensor is `tensor` (batch, channel, height, width)
tensor = torch.randn(500,60, 224, 224)

# Create an HDF5 file
with h5py.File('tensor_data.h5', 'w') as hf:
    # Create a dataset within the HDF5 file
    hf.create_dataset('data', data=tensor.numpy())


In [1]:
import h5py

# Ouverture du fichier HDF5 en mode lecture
with h5py.File('tensor_data.h5', 'r') as f:
    # Accéder au jeu de données
    dataset = f["./"]
    dataset_2= f["./"]
    dataset_3= f["./"]
    dataset_4= f["./"]

In [2]:
dataset.shape

AttributeError: 'Group' object has no attribute 'shape'